In [1]:
from models.lbanp import LBANP
from models.tnpnd import TNPND

In [2]:
lbanp_model = LBANP(num_latents=8,
                dim_x= 2,
                dim_y= 5,
                d_model=16,
                emb_depth=3,
                dim_feedforward=64,
                nhead=1,
                dropout=0.0,
                num_layers=4,
                drop_y=0.5,
                device="cpu")

tnp_model = TNPND(
        dim_x= 2,
                dim_y= 5,
                d_model=16,
                emb_depth=3,
                dim_feedforward=64,
                nhead=1,
                dropout=0.0,
                num_layers=4,
                num_std_layers=2,
                drop_y=0.5,
                cov_approx='lowrank', # cholesky or lowrank parameterization
                prj_dim=20,
                prj_depth=4,
                diag_depth=4,
    device="cpu"
    )

In [3]:
from data.wheel import sample_wheel_data, WheelBanditSampler

In [4]:
sampler = WheelBanditSampler()

In [5]:
batch = sampler.sample(
            batch_size=8,
            device="cpu",
            seed=0,
            reward="all"
        )

In [6]:
from inference.ar import ar_predict, ar_predict_for_loop
import numpy as np
import torch
generator = torch.Generator().manual_seed(0)
mean, var, yt, ft = ar_predict(generator, lbanp_model, [batch.xc], [batch.yc], [batch.xt], num_samples=3)

buckets torch.Size([3, 8, 1, 5])
buckets2 torch.Size([3, 8, 1, 5])
torch.Size([8, 50, 5])
buckets torch.Size([8, 1, 5])
buckets2 torch.Size([8, 1, 5])
buckets torch.Size([8, 1, 5])
buckets2 torch.Size([8, 1, 5])


In [7]:
mean[0].shape

torch.Size([8, 50, 5])

In [13]:
ft[0]

tensor([[[[ 1.2676e-02,  1.9557e-01, -4.7441e-01, -3.2841e-03,  8.8925e-02],
          [ 1.5165e-02,  2.0311e-01, -4.5259e-01, -1.2500e-03,  8.9572e-02],
          [ 3.0848e-02,  1.5028e-01, -5.2930e-01,  4.4914e-02,  1.4377e-02],
          ...,
          [ 1.4319e-02,  1.7075e-01, -4.5740e-01,  1.4142e-02,  8.6010e-02],
          [ 4.6377e-02,  1.6353e-01, -4.8324e-01,  5.8060e-02,  5.2288e-02],
          [ 7.4214e-03,  1.8797e-01, -4.5160e-01, -1.8470e-04,  9.1068e-02]],

         [[ 1.3468e-01,  1.1158e-01, -4.3490e-01,  1.3946e-01,  4.5757e-02],
          [ 7.5138e-02,  1.4588e-01, -3.1452e-01,  1.0223e-01,  8.8063e-02],
          [ 7.0974e-02,  1.3296e-01, -3.0084e-01,  1.0948e-01,  8.9681e-02],
          ...,
          [ 6.1766e-02,  1.4524e-01, -2.9832e-01,  9.3956e-02,  9.7708e-02],
          [ 1.0282e-01,  1.1883e-01, -3.6184e-01,  1.3170e-01,  6.9598e-02],
          [ 7.8225e-02,  1.2203e-01, -3.4602e-01,  1.1301e-01,  7.5388e-02]],

         [[ 3.7526e-02,  2.0216e-01, -5.91

In [9]:
batch.y.shape

torch.Size([8, 562, 5])

In [10]:
batch.y[0][0]

tensor([1.1835, 0.9955, 0.9711, 1.0368, 0.9504])

In [11]:
batch.xc.shape

torch.Size([8, 512, 2])

In [12]:
batch.yc.shape

torch.Size([8, 512, 5])

In [13]:
batch.xt.shape

torch.Size([8, 50, 2])

In [14]:
batch.yt.shape

torch.Size([8, 50, 5])

In [7]:
ll_loss, pred = lbanp_model(batch)

In [14]:
tnp_model(batch)

AttrDict({'loss_target': tensor(666.4536, grad_fn=<NegBackward0>), 'loss_ctx': tensor(1781.4780, grad_fn=<MeanBackward0>), 'loss': tensor(2447.9316, grad_fn=<AddBackward0>), 'mean_std': tensor(0.0412, grad_fn=<MeanBackward0>), 'rmse': tensor(336.7043, grad_fn=<MeanBackward0>)})

In [17]:
pred.sample(torch.Size([10]))[1]

tensor([[[-0.4370, -1.0098,  0.1075, -0.9991,  0.4572],
         [-0.1563, -0.3379,  0.4788,  0.1298,  0.6714],
         [ 0.2853, -0.2805,  0.3360,  0.8649, -0.9022],
         ...,
         [ 1.2777,  0.1652,  0.6480,  0.2400,  1.1537],
         [ 0.7377,  0.2597,  1.0902, -0.0205, -0.4204],
         [ 0.1903, -0.3400,  0.1030, -0.1797,  0.2685]],

        [[-0.3725,  0.3566, -1.8452,  0.7515,  0.4054],
         [ 0.6287, -0.5361,  1.3953,  1.1158, -0.8499],
         [-0.5555, -0.6780,  3.5492,  0.5310,  1.0871],
         ...,
         [ 1.2121,  0.1738,  2.3070, -1.4268,  1.1481],
         [ 1.1972, -0.1266,  0.5311, -0.2256,  0.8538],
         [ 1.8189, -0.7754, -0.6902, -0.3359, -0.3103]],

        [[ 0.8968,  0.4521,  0.4219, -1.3565, -0.3342],
         [-1.7084,  0.4685,  1.7808,  0.9149,  0.6502],
         [ 1.3056, -1.0105, -0.4376,  0.3604,  2.1575],
         ...,
         [ 0.1437, -1.3362, -0.9561,  0.4158,  0.5472],
         [ 1.8388, -0.7121, -1.5469, -0.1130, -0.5621],
  

In [23]:
batch.yc.shape

torch.Size([8, 512, 5])

In [8]:
import numpy as np
a = np.random.randn(3,2,2)

In [9]:
a

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])

In [10]:
a.reshape(-1,2).reshape(3,2,2)

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])